<a href="https://colab.research.google.com/gist/oserikov/f430e81939ffff48cafd6377b9e67b9c/.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание о векторизации текстов

В задании вам предстоит сравнить несколько методов снижения размерности
* PCA
* t-SNE

а так же попробовать осуществить тематическое моделирование методом LDA.

**Формат сдачи задания** -- указание в гуглформе ссылки на тетрадь с решением + ответ на вопросы (см. последние вопросы первой задачи) в форме. Форма появится ближе к дедлайну.

**Дедлайн** 23.59 7 октября MSK.  

ДЗ предполагает возможность получения **до 12 баллов** по десятибалльной шкале. Оценки 11 и 12 поступают в ведомость, как оценки 11 и 12.

---

Если вы уже хорошо знакомы с снижением размерности, реализуйте первую задачу, используя не Bag-of-Words векторы текстов, а эмбеддинги текстов, полученные алгоритмом на ваш выбор. 
**Если вы собираетесь решать задачу так, то, приступая, сообщите об этом @oserikov в телеграме.**

Если вы уже хорошо знакомы ещё и с векторизацией текстов эмбеддингами, напишите @oserikov для обсуждения замены первой задачи на другую.


# [6 баллов] Задача о снижении размерности




[Вот](https://drive.google.com/drive/folders/1HX5rz4UZHtbzhPguUFolOg-xm6HFc0KO?usp=sharing) корпус, однажды собранный без особенных размышлений.
Это -- корпус любительской литературы. Он был собран для забавы и непонятно, какая природа у представленных там текстов.

Вам предстоит оценить, насколько эти тексты интересны в качестве простого датасета для задачи классификации: информативны ли Bag-of-Words векторы в смысле разделения текстов по жанрам.

---

### Постановка задачи

**Задача**: взяв фанифики и два каких-то других жанра из корпуса, визуализировать их BoW-представления на плоскости.

---

Визуализацию стоит осуществлять scatter-плотом, информацию о принадлежности документа какому-то жанру стоит передавать цветом.

Количество документов, представляющих каждый жанр, стоит подобрать семплированием нужного количества элементов под доступные вычислительные ресурсы -- полный корпус точно слишком велик.

Гиперпараметры BoW-векторизатора стоит подобрать под доступные вычислительные ресурсы -- если код работает дольше часа, то стоит упростить вычислительную задачу: подобрать другие гиперпараметры векторизации или уменьшить выборку.

#### Критерии


* **1 БАЛЛ**: 
  * В выбранных документах осуществлена какая-то **стандартная предобработка текста**: удалены стоп-слова и мусорные токены (e.g. html-теги), проведена лемматизация.  
  Решение о каждой конкретной детали предобработки остаётся на усмотрение студентов: каждое нестандартное действие (e.g. отказ от лемматизации или удаление каких-то особенных токенов) стоит пояснить коротким комментарием, описывающим мотивацию.
  * Получены **Bag-of-Words векторы** документов, выбранных для исследования. 
* **1 БАЛЛ**: получена визуализация документов на плоскости **методом главных компонент** снижения размерности Bag-of-Words векторов.
* **1 БАЛЛ**: получена визуализация документов на плоскости методом **t-SNE** снижения размерности Bag-of-Words векторов.
* **1 БАЛЛ**: на полученных визуализациях **получилось передать цветом точек классы** документов; понятно, точка какого цвета относится к какому классу.


Скорее всего визуализация t-SNE и PCA заметно отличаются раскладкой точек по плоскости: один метод как будто раскладывает их вдоль двух пересекающихся прямых, за другим такого свойства скорее всего нет. Ответ на два вопроса ниже вам предстоит указать в гуглформе, сдавая задание.
* **1 БАЛЛ**: верно указано, какой метод укладывает точки примерно вдоль прямых, а какой -- нет
* **1 БАЛЛ**: предложено верное описание тому, почему у одного из методов всегда результаты располагаются вдоль некоторых прямых. 



#### Примеры кода

Использование t-SNE и PCA для визуализации векторов: [ссылка](https://www.kaggle.com/jbencina/clustering-documents-with-tfidf-and-kmeans).





# Решение задачи

## Стандартная предобработка текстов

In [8]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words("russian"))

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

import re

def tokenize (text):
    text = text.lower()
    text = re.sub(r'[.!?]\s', r' ', text)
    words = text.split()
    clean_words = []
    for word in words:
        if word not in stop_words:
            clean_words.append(word)
    return clean_words

def lemmatize (text):
    words = tokenize(text)
    lemmas = []
    for word in words:
        lemma = morph.parse(word)[0].normal_form
        lemmas.append(lemma)
        new_lemmas = ' '.join(lemmas)        
    return new_lemmas

In [9]:
import os

num_of_texts = 5

folders = ['scifi', 'fanfiction', 'esoterics']
names = []
texts = []
genres = []

new_folders = []
for folder in folders:
    new_folders.append (os.walk(folder))

for folder in new_folders:
    for address, dirs, files in folder:
        i = 0
        for file in files:
            names.append(file)
            path = address+'/'+file
            opened = open(path, 'r', encoding = 'utf-8')
            text = opened.read()
            texts.append(text)
            genres.append(address)
            i += 1
            if i > num_of_texts - 1:
                break

In [10]:
cleaned = []
for text in texts:
    text = lemmatize(text)
    cleaned.append(text)

In [11]:
import pandas as pd

df = pd.DataFrame()

df['genre'] = genres
df['name'] = names
df['texts'] = texts
df['lemmas'] = cleaned

In [12]:
df

,genre,name,texts,lemmas
0,scifi,makar_dmyeshepowojuem.txt,"\nЧто-то стучится в висок. То ли кровь, то ли ...","что-то стучаться висок кровь, недавний выстрел..."
1,scifi,maks_roudi.txt,\n\n\n\n \n\n МАКС РОУД љ\n\n\n 05.11.2...,макс роуд љ 05.11.2014 - 03.04.2015 агония маз...
2,scifi,mak_ivanzzz_ideal3.txt,\n\n\n\n\n\n\n\n\n Ivan Mak\n Идеальный мир\n\...,ivan mak идеальный мир предисловие медленно по...
3,scifi,malaja_m_s16iacmoguch21.txt,\n\n\n\n\nГлава 15.\n\n***\n\n4403 цикл Космич...,глава 15 *** 4403 цикл космический эра планет ...
4,scifi,malinowskaja_majja_igorewnaplennikiuest.txt,\n\n\n \n \n\n\n\n\n\nМалиновкая Майя\n\n\n\n\...,малиновкий майя пленник уэст книга 2 фантастич...
5,fanfiction,kowalenko_e_blentochka2.txt,\nЛенточка \n\n Фанфик по роману ...,"ленточка фанфик роман андрей круз ""земля лишни..."
6,fanfiction,kowizhenko_w_wmasseffectwhilethereaperukr.txt,"\n"" Mass Effect : While the Reaper ""\n\nНазва:...",""" mass effect : while the reaper "" назва: "" ma..."
7,fanfiction,kram_dhivepodkidyshiulxja.txt,\n\nДмитрий Крам.\nS-T-I-K-S. Подкидыши Улья.\...,дмитрий крам s-t-i-k-s подкидыш улья андрей пр...
8,fanfiction,kram_discheznuwshijklan.txt,\n\n\nДмитрий Крам.\n\nИсчезнувший клан. Фанфи...,"дмитрий крам исчезнуть клан фанфик ""играть жит..."
9,fanfiction,krasnoperowa_ahp_peste.txt,\n\n\t\n\t\t\n\t\t\t\n\t\t\t\tСодержание:\n\t\...,содержание: часть 1 акклиматиазия часть 2 полё...


## Bag-of-Words векторы

In [39]:
corpus = df['lemmas'].values.tolist()

In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

## Визуализация

In [50]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm

from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

## [6 баллов] Задача о тематическом моделировании



### об LDA




### Постановка задачи
Загрузите [коллекцию писем Х. Клинтон](https://www.kaggle.com/kaggle/hillary-clinton-emails/?select=Emails.csv) с kaggle. Для скачивания может потребоваться регистрация.

Методом LDA выделите несколько тем в переписке Х. Клинтон, дайте им словесное описание. Используйте библиотеку LdaModel из gensim.

#### Критерии

* **2 БАЛЛА**: получены списки ключевых слов, не выглядящие бессмыслицей
* **2 БАЛЛА**: осуществлена визуализация библиотекой pyLDAvis
* **1 БАЛЛ**: предложено осмысленное текстовое описание большинства выделенных тем.
* **1 БАЛЛ**: проведено сравнение LDA, запущенного на CountVectorizer и TfIdfVectorizer предтавлениях одних и тех же данных. 

#### примеры кода

Пример обучения LdaModel на выдаче CountVectorizer: [ссылка](https://github.com/EricSchles/sklearn_gensim_example/blob/master/example.py)

Пример использования pyLDAvis: секция 15 [по ссылке](https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/)

---

Для обучения *LdaModel* и её последующей визуализации потребуется словарь формата gensim. Словарь формата gensim удобно получать из сжатого csc_matrix-представления нашего векторизованного текста: как многие замечали на паре, tf-idf векторы содержат много нулей.

```python
import gensim
from scipy.sparse import csc

corpus = gensim.matutils.Sparse2Corpus(csc.csc_matrix(X))
dictionary = gensim.corpora.Dictionary.from_corpus(corpus, vocab_dict)
```

где *corpora* содержит полученное с помощью gensim представление коллекции, а *vocab_dict* — это dict, полученный после работы Vectorizer, ставящий в соответствие каждому номеру строки в матрице данных само слово в виде строки.